In [ ]:
Northwest Course Catlog Storing

In [ ]:
# ============================================================
# Northwest Missouri State University Catalog Scraper
# Purpose: Scrape course catalog and store in Pinecone for RAG
# ============================================================

# Step 1: Install required packages
!pip install -q requests beautifulsoup4 openai pinecone-client lxml
!pip uninstall -y pinecone-client
!pip install -U pinecone

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs
import json
import time
import re
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from collections import deque
import hashlib

# ============================================================
# Enhanced Catalog Scraper Class
# ============================================================
class CatalogScraper:
    def __init__(self, base_url, max_depth=3):
        self.base_url = base_url
        self.max_depth = max_depth
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        })
        self.visited_urls = set()
        self.all_data = []
        self.base_domain = urlparse(base_url).netloc

    def is_valid_internal_link(self, url):
        """Check if URL is a valid internal link"""
        try:
            parsed = urlparse(url)

            # Must be from same domain
            if parsed.netloc and parsed.netloc != self.base_domain:
                return False

            # Exclude file downloads
            excluded_extensions = ['.pdf', '.doc', '.docx', '.xls', '.xlsx',
                                  '.zip', '.jpg', '.png', '.gif', '.mp4']
            if any(url.lower().endswith(ext) for ext in excluded_extensions):
                return False

            # Exclude external links and non-content URLs
            excluded_patterns = ['mailto:', 'tel:', 'javascript:', '#',
                               'facebook.com', 'twitter.com', 'instagram.com',
                               'youtube.com', 'linkedin.com']
            if any(pattern in url.lower() for pattern in excluded_patterns):
                return False

            # Focus on catalog content
            if 'catalog.nwmissouri.edu' in url:
                # Include content pages, course pages, program pages
                valid_patterns = ['content.php', 'courses.php', 'preview_program.php',
                                'preview_course.php', 'catoid=', 'navoid=']
                return any(pattern in url for pattern in valid_patterns)

            return False

        except Exception as e:
            print(f"Error validating URL {url}: {e}")
            return False

    def normalize_url(self, url):
        """Normalize URL to avoid duplicates"""
        parsed = urlparse(url)
        # Remove fragments and normalize query parameters
        query_params = parse_qs(parsed.query)
        # Sort parameters for consistency
        sorted_query = '&'.join(f"{k}={v[0]}" for k, v in sorted(query_params.items()))
        normalized = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
        if sorted_query:
            normalized += f"?{sorted_query}"
        return normalized

    def extract_links_from_page(self, url, soup):
        """Extract all valid internal links from a page"""
        links = set()

        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urljoin(url, href)
            normalized_url = self.normalize_url(full_url)

            if (self.is_valid_internal_link(normalized_url) and
                normalized_url not in self.visited_urls):
                links.add(normalized_url)

        return links

    def clean_text(self, text):
        """Clean and normalize text content"""
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        # Remove multiple newlines
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()

    def extract_page_content(self, url):
        """Extract structured content from a catalog page"""
        try:
            response = self.session.get(url, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove unwanted elements
            for element in soup(['script', 'style', 'nav', 'header',
                               'footer', 'aside', 'iframe']):
                element.decompose()

            # Extract title
            title = soup.find('title')
            title_text = title.get_text(strip=True) if title else 'No Title'

            # Try to find main content area (catalog-specific selectors)
            main_content = (
                soup.find('div', id='content') or
                soup.find('div', class_='block') or
                soup.find('td', class_='block_content') or
                soup.find('main') or
                soup.find('article') or
                soup.body
            )

            if not main_content:
                return None

            # Extract text content
            text_content = main_content.get_text(separator='\n', strip=True)
            text_content = self.clean_text(text_content)

            # Extract structured information for courses/programs
            structured_data = self.extract_structured_info(soup, url)

            # Extract all links for crawling
            links = self.extract_links_from_page(url, soup)

            return {
                'url': url,
                'title': title_text,
                'content': text_content,
                'structured_data': structured_data,
                'content_length': len(text_content),
                'links': links
            }

        except Exception as e:
            print(f"❌ Error extracting content from {url}: {e}")
            return None

    def extract_structured_info(self, soup, url):
        """Extract structured information like course details, program info"""
        structured = {
            'type': 'general',
            'details': {}
        }

        # Detect content type from URL
        if 'preview_course' in url:
            structured['type'] = 'course'
            structured['details'] = self.extract_course_info(soup)
        elif 'preview_program' in url:
            structured['type'] = 'program'
            structured['details'] = self.extract_program_info(soup)
        elif 'courses.php' in url:
            structured['type'] = 'course_list'

        return structured

    def extract_course_info(self, soup):
        """Extract course-specific information"""
        course_info = {}

        # Try to find course title
        course_title = soup.find('h1') or soup.find('h2')
        if course_title:
            course_info['title'] = course_title.get_text(strip=True)

        # Try to find course code
        course_code = soup.find(text=re.compile(r'[A-Z]{3,4}\s*\d{3,4}'))
        if course_code:
            course_info['code'] = course_code.strip()

        # Try to find credit hours
        credits = soup.find(text=re.compile(r'\d+\s*credit', re.IGNORECASE))
        if credits:
            course_info['credits'] = credits.strip()

        return course_info

    def extract_program_info(self, soup):
        """Extract program-specific information"""
        program_info = {}

        # Try to find program title
        program_title = soup.find('h1') or soup.find('h2')
        if program_title:
            program_info['title'] = program_title.get_text(strip=True)

        # Try to find degree type
        degree = soup.find(text=re.compile(r'Bachelor|Master|Associate|Certificate', re.IGNORECASE))
        if degree:
            program_info['degree'] = degree.strip()

        return program_info

    def crawl_bfs(self, start_url):
        """Breadth-First Search crawling with depth limit"""
        queue = deque([(start_url, 0)])  # (url, depth)
        self.visited_urls.add(self.normalize_url(start_url))

        print("🚀 Starting BFS crawl...")
        print(f"📊 Max depth: {self.max_depth}")

        while queue:
            current_url, depth = queue.popleft()

            if depth > self.max_depth:
                continue

            print(f"\n{'  ' * depth}📄 [Depth {depth}] Crawling: {current_url[:80]}...")

            # Extract content from current page
            page_data = self.extract_page_content(current_url)

            if page_data and page_data['content_length'] > 100:
                # Store the page data
                self.all_data.append({
                    'id': len(self.all_data),
                    'url': page_data['url'],
                    'title': page_data['title'],
                    'content': page_data['content'],
                    'content_length': page_data['content_length'],
                    'structured_data': page_data['structured_data'],
                    'depth': depth,
                    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
                })

                print(f"{'  ' * depth}✅ Extracted {page_data['content_length']} chars")
                print(f"{'  ' * depth}🔗 Found {len(page_data['links'])} links")

                # Add new links to queue if not at max depth
                if depth < self.max_depth:
                    for link in page_data['links']:
                        normalized_link = self.normalize_url(link)
                        if normalized_link not in self.visited_urls:
                            self.visited_urls.add(normalized_link)
                            queue.append((link, depth + 1))

            # Rate limiting
            time.sleep(0.5)

        print(f"\n✅ Crawling completed!")
        print(f"📄 Total pages scraped: {len(self.all_data)}")
        return self.all_data

    def save_to_jsonl(self, data, filename="catalog_data.jsonl"):
        """Save scraped data to JSONL file"""
        with open(filename, 'w', encoding='utf-8') as f:
            for item in data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f"💾 Data saved to {filename}")

# ============================================================
# Pinecone Integration
# ============================================================
class PineconeRAG:
    def __init__(self, openai_api_key, pinecone_api_key, index_name="nw-catalog-rag"):
        self.client = OpenAI(api_key=openai_api_key)
        self.pc = Pinecone(api_key=pinecone_api_key)
        self.index_name = index_name
        self.index = None

    def create_or_connect_index(self, dimension=1536):
        """Create new index or connect to existing one"""
        try:
            # Try to connect to existing index
            self.index = self.pc.Index(self.index_name)
            print(f"✅ Connected to existing index: {self.index_name}")
        except:
            print(f"📝 Creating new index: {self.index_name}")
            self.pc.create_index(
                name=self.index_name,
                dimension=dimension,
                metric="cosine",
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1"
                )
            )
            time.sleep(10)  # Wait for index to initialize
            self.index = self.pc.Index(self.index_name)
            print(f"✅ Index created successfully")

    def chunk_text(self, text, chunk_size=1000, overlap=200):
        """Split text into overlapping chunks"""
        chunks = []
        start = 0

        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]

            # Try to break at sentence boundary
            if end < len(text):
                last_period = chunk.rfind('.')
                last_newline = chunk.rfind('\n')
                break_point = max(last_period, last_newline)

                if break_point > chunk_size * 0.5:  # At least 50% into chunk
                    chunk = chunk[:break_point + 1]
                    end = start + break_point + 1

            chunks.append(chunk.strip())
            start = end - overlap

        return chunks

    def generate_embedding(self, text):
        """Generate embedding using OpenAI"""
        try:
            response = self.client.embeddings.create(
                model="text-embedding-3-small",
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"❌ Error generating embedding: {e}")
            return None

    def insert_documents(self, documents, batch_size=100):
        """Insert documents into Pinecone with chunking"""
        print(f"\n🗄️ Inserting {len(documents)} documents into Pinecone...")

        successful_inserts = 0
        vectors_to_upsert = []

        for doc_idx, doc in enumerate(documents):
            try:
                # Chunk the content
                chunks = self.chunk_text(doc['content'])

                for chunk_idx, chunk in enumerate(chunks):
                    if len(chunk.strip()) < 50:  # Skip very small chunks
                        continue

                    # Prepare text for embedding
                    embedding_text = f"""
Title: {doc['title']}
Type: {doc['structured_data']['type']}
Content: {chunk}
URL: {doc['url']}
                    """.strip()

                    # Generate embedding
                    embedding = self.generate_embedding(embedding_text)

                    if embedding:
                        # Create unique ID
                        unique_id = f"doc_{doc_idx}_chunk_{chunk_idx}_{hashlib.md5(chunk.encode()).hexdigest()[:8]}"

                        # Prepare metadata
                        metadata = {
                            'title': doc['title'][:500],  # Limit length
                            'url': doc['url'],
                            'content': chunk[:1000],  # Store chunk preview
                            'type': doc['structured_data']['type'],
                            'depth': doc['depth'],
                            'chunk_index': chunk_idx,
                            'total_chunks': len(chunks),
                            'timestamp': doc['timestamp']
                        }

                        # Add structured data if available
                        if doc['structured_data']['details']:
                            for key, value in doc['structured_data']['details'].items():
                                if isinstance(value, str) and len(value) < 500:
                                    metadata[f'detail_{key}'] = value

                        vectors_to_upsert.append({
                            'id': unique_id,
                            'values': embedding,
                            'metadata': metadata
                        })

                        successful_inserts += 1

                # Batch upsert
                if len(vectors_to_upsert) >= batch_size:
                    self.index.upsert(vectors=vectors_to_upsert)
                    print(f"  ✅ Inserted batch of {len(vectors_to_upsert)} vectors")
                    vectors_to_upsert = []

                if (doc_idx + 1) % 10 == 0:
                    print(f"  📊 Processed {doc_idx + 1}/{len(documents)} documents...")

            except Exception as e:
                print(f"  ❌ Error processing document {doc_idx}: {e}")
                continue

        # Insert remaining vectors
        if vectors_to_upsert:
            self.index.upsert(vectors=vectors_to_upsert)
            print(f"  ✅ Inserted final batch of {len(vectors_to_upsert)} vectors")

        print(f"\n✅ Total vectors inserted: {successful_inserts}")
        return successful_inserts

# ============================================================
# Main Execution
# ============================================================
if __name__ == "__main__":
    print("=" * 60)
    print("Northwest Missouri State University Catalog Scraper")
    print("=" * 60)

    # Configuration
    BASE_URL = "https://catalog.nwmissouri.edu/content.php?catoid=6&navoid=177"
    MAX_DEPTH = 3

    # API Keys - REPLACE WITH YOUR ACTUAL KEYS
    OPENAI_API_KEY = ""
    PINECONE_API_KEY = ""
    INDEX_NAME = "salesassistai"

    # Validate API keys
    if "your-" in OPENAI_API_KEY or "your-" in PINECONE_API_KEY:
        print("⚠️  WARNING: Please replace the API keys with your actual keys!")
        print("Update OPENAI_API_KEY and PINECONE_API_KEY variables.")
    else:
        # Step 1: Scrape the catalog
        scraper = CatalogScraper(BASE_URL, max_depth=MAX_DEPTH)
        scraped_data = scraper.crawl_bfs(BASE_URL)

        # Save raw data
        scraper.save_to_jsonl(scraped_data, "catalog_raw_data.jsonl")

        # Step 2: Insert into Pinecone
        rag = PineconeRAG(OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME)
        rag.create_or_connect_index()

        successful_inserts = rag.insert_documents(scraped_data)

        # Final Summary
        print("\n" + "=" * 60)
        print("📊 FINAL SUMMARY")
        print("=" * 60)
        print(f"✅ Total pages scraped: {len(scraped_data)}")
        print(f"✅ Total vectors inserted: {successful_inserts}")
        print(f"✅ Index name: {INDEX_NAME}")
        print(f"✅ Raw data saved: catalog_raw_data.jsonl")
        print("=" * 60)

Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


Northwest Missouri State University Catalog Scraper
🚀 Starting BFS crawl...
📊 Max depth: 3

📄 [Depth 0] Crawling: https://catalog.nwmissouri.edu/content.php?catoid=6&navoid=177...
✅ Extracted 15841 chars
🔗 Found 270 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/search_advanced.php?catoid=6...
  ✅ Extracted 499 chars
  🔗 Found 0 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1420&returnto=1...


/tmp/ipython-input-3592868707.py:209: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  degree = soup.find(text=re.compile(r'Bachelor|Master|Associate|Certificate', re.IGNORECASE))


  ✅ Extracted 5078 chars
  🔗 Found 1 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1516&returnto=1...
  ✅ Extracted 2197 chars
  🔗 Found 3 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1330&returnto=1...
  ✅ Extracted 1047 chars
  🔗 Found 2 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1233&returnto=1...
  ✅ Extracted 1831 chars
  🔗 Found 8 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1236&returnto=1...
  ✅ Extracted 15673 chars
  🔗 Found 13 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/content.php?catoid=6&navoid=167...
  ✅ Extracted 74552 chars
  🔗 Found 0 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1218&returnto=1...
  ✅ Extracted 2090 chars
  🔗 Found 0 links

  📄 [Depth 1] Crawling: https://catalog.nwmissouri.edu/prev

/tmp/ipython-input-3592868707.py:188: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  course_code = soup.find(text=re.compile(r'[A-Z]{3,4}\s*\d{3,4}'))
/tmp/ipython-input-3592868707.py:193: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  credits = soup.find(text=re.compile(r'\d+\s*credit', re.IGNORECASE))


Streaming output truncated to the last 5000 lines.
    📄 [Depth 2] Crawling: https://catalog.nwmissouri.edu/preview_course_nopop.php?catoid=6&coid=8698...
    ✅ Extracted 735 chars
    🔗 Found 0 links

    📄 [Depth 2] Crawling: https://catalog.nwmissouri.edu/preview_course_nopop.php?catoid=6&coid=9217...
    ✅ Extracted 1029 chars
    🔗 Found 1 links

    📄 [Depth 2] Crawling: https://catalog.nwmissouri.edu/preview_course_nopop.php?catoid=6&coid=9050...
    ✅ Extracted 1174 chars
    🔗 Found 0 links

    📄 [Depth 2] Crawling: https://catalog.nwmissouri.edu/preview_course_nopop.php?catoid=6&coid=7868...
    ✅ Extracted 1026 chars
    🔗 Found 0 links

    📄 [Depth 2] Crawling: https://catalog.nwmissouri.edu/preview_course_nopop.php?catoid=6&coid=9049...
    ✅ Extracted 1319 chars
    🔗 Found 0 links

    📄 [Depth 2] Crawling: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1391...
    ✅ Extracted 2158 chars
    🔗 Found 0 links

    📄 [Depth 2] Crawling: https://catalog.n

Testing RAG For Pipeline

In [ ]:
# ============================================================
# Test Script for Pinecone RAG System
# Purpose: Query and validate responses from the stored catalog data
# ============================================================

from openai import OpenAI
from pinecone import Pinecone
import json
from typing import List, Dict

# ============================================================
# RAG Query System
# ============================================================
class CatalogRAGTester:
    def __init__(self, openai_api_key: str, pinecone_api_key: str, index_name: str = "salesassistai"):
        """Initialize the RAG tester with API keys"""
        self.client = OpenAI(api_key=openai_api_key)
        self.pc = Pinecone(api_key=pinecone_api_key)
        self.index = self.pc.Index(index_name)
        self.index_name = index_name

    def generate_embedding(self, text: str) -> List[float]:
        """Generate embedding for a query"""
        try:
            response = self.client.embeddings.create(
                model="text-embedding-3-small",
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"❌ Error generating embedding: {e}")
            return None

    def search_similar(self, query: str, top_k: int = 5) -> List[Dict]:
        """Search for similar documents in Pinecone"""
        try:
            # Generate query embedding
            query_embedding = self.generate_embedding(query)

            if not query_embedding:
                return []

            # Search in Pinecone
            results = self.index.query(
                vector=query_embedding,
                top_k=top_k,
                include_metadata=True
            )

            return results['matches']
        except Exception as e:
            print(f"❌ Error searching Pinecone: {e}")
            return []

    def generate_answer(self, query: str, context: str) -> str:
        """Generate answer using OpenAI with retrieved context"""
        try:
            prompt = f"""You are a helpful assistant for Northwest Missouri State University's academic catalog.

Use the following context from the university catalog to answer the question.
If the answer cannot be found in the context, say "I don't have enough information to answer that question."

Context:
{context}

Question: {query}

Answer:"""

            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful university catalog assistant. Provide accurate, concise answers based on the given context."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=500
            )

            return response.choices[0].message.content
        except Exception as e:
            print(f"❌ Error generating answer: {e}")
            return "Error generating response."

    def query(self, question: str, top_k: int = 5, show_sources: bool = True) -> Dict:
        """Complete RAG query pipeline"""
        print(f"\n{'='*80}")
        print(f"🔍 QUESTION: {question}")
        print(f"{'='*80}")

        # Step 1: Search for relevant documents
        print("\n📚 Searching for relevant information...")
        matches = self.search_similar(question, top_k=top_k)

        if not matches:
            return {
                "question": question,
                "answer": "No relevant information found.",
                "sources": []
            }

        print(f"✅ Found {len(matches)} relevant documents")

        # Step 2: Extract context from matches
        context_parts = []
        sources = []

        for i, match in enumerate(matches):
            score = match['score']
            metadata = match.get('metadata', {})

            # Extract relevant info from metadata
            title = metadata.get('title', 'Unknown')
            content = metadata.get('content', '')
            url = metadata.get('url', '')
            doc_type = metadata.get('type', 'general')

            context_parts.append(f"[Source {i+1}] {content}")
            sources.append({
                'title': title,
                'url': url,
                'type': doc_type,
                'relevance_score': round(score, 4)
            })

            if show_sources:
                print(f"\n  📄 Source {i+1}:")
                print(f"     Title: {title[:80]}...")
                print(f"     Type: {doc_type}")
                print(f"     Relevance: {score:.4f}")
                print(f"     URL: {url[:100]}...")

        # Combine context
        context = "\n\n".join(context_parts)

        # Step 3: Generate answer
        print(f"\n🤖 Generating answer using GPT-3.5...")
        answer = self.generate_answer(question, context)

        print(f"\n{'='*80}")
        print(f"💡 ANSWER:")
        print(f"{'='*80}")
        print(answer)
        print(f"{'='*80}\n")

        return {
            "question": question,
            "answer": answer,
            "sources": sources,
            "num_sources": len(sources)
        }

    def batch_test(self, questions: List[str], show_sources: bool = False) -> List[Dict]:
        """Test multiple questions at once"""
        results = []

        print(f"\n{'#'*80}")
        print(f"# BATCH TESTING - {len(questions)} Questions")
        print(f"{'#'*80}\n")

        for i, question in enumerate(questions, 1):
            print(f"\n{'>'*80}")
            print(f"> Test {i}/{len(questions)}")
            print(f"{'>'*80}")

            result = self.query(question, show_sources=show_sources)
            results.append(result)

            # Brief pause between queries to avoid rate limits
            import time
            time.sleep(0.5)

        return results

    def print_summary(self, results: List[Dict]):
        """Print summary of test results"""
        print(f"\n{'#'*80}")
        print(f"# TEST SUMMARY")
        print(f"{'#'*80}\n")

        for i, result in enumerate(results, 1):
            print(f"{i}. Question: {result['question'][:70]}...")
            print(f"   Sources Found: {result['num_sources']}")
            print(f"   Answer Preview: {result['answer'][:100]}...")
            print()


# ============================================================
# Test Questions
# ============================================================
TEST_QUESTIONS = [
    # Program/Major Related
    "What computer science programs are available at Northwest Missouri State University?",
    "Tell me about the Business Administration degree requirements",
    "What are the admission requirements for graduate programs?",
    "Does the university offer any engineering programs?",

    # Course Related
    "What courses are required for a Bachelor's in Psychology?",
    "Are there any online degree programs available?",
    "What mathematics courses does the university offer?",

    # General Academic
    "What is the GPA requirement for graduation?",
    "How many credit hours are needed to complete a bachelor's degree?",
    "What minors are available for students?",

    # Specific Department Queries
    "What classes are in the English department?",
    "Tell me about the Education programs",
    "What nursing programs does NW Missouri offer?",

    # Requirements and Policies
    "What are the general education requirements?",
    "Are internships required for any programs?",
    "What is the tuition and fees structure?",
]


# ============================================================
# Main Execution
# ============================================================
if __name__ == "__main__":
    print("=" * 80)
    print("Northwest Missouri State University - Catalog RAG Testing")
    print("=" * 80)

    # Configuration - REPLACE WITH YOUR ACTUAL API KEYS
    OPENAI_API_KEY = ""
    PINECONE_API_KEY = ""
    INDEX_NAME = "salesassistai"

    # Validate API keys
    if "your-" in OPENAI_API_KEY or "your-" in PINECONE_API_KEY:
        print("\n⚠️  WARNING: Please replace the API keys with your actual keys!")
        print("Update OPENAI_API_KEY and PINECONE_API_KEY variables.")
        exit(1)

    # Initialize tester
    print("\n🔧 Initializing RAG Tester...")
    tester = CatalogRAGTester(OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME)
    print(f"✅ Connected to Pinecone index: {INDEX_NAME}\n")

    # Option 1: Test a single question with detailed output
    print("\n" + "="*80)
    print("OPTION 1: Single Question Test (Detailed)")
    print("="*80)

    single_result = tester.query(
        "What computer science programs are available?",
        top_k=3,
        show_sources=True
    )

    # Option 2: Batch test multiple questions
    print("\n" + "="*80)
    print("OPTION 2: Batch Test (Multiple Questions)")
    print("="*80)

    # Select a subset of questions for testing
    selected_questions = TEST_QUESTIONS[:5]  # Test first 5 questions

    batch_results = tester.batch_test(selected_questions, show_sources=False)

    # Print summary
    tester.print_summary(batch_results)

    # Option 3: Custom interactive query
    print("\n" + "="*80)
    print("OPTION 3: Interactive Query Mode")
    print("="*80)
    print("\nYou can now ask custom questions. Type 'quit' to exit.\n")

    while True:
        user_question = input("❓ Your question: ").strip()

        if user_question.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye!")
            break

        if not user_question:
            continue

        tester.query(user_question, top_k=3, show_sources=True)

    print("\n" + "="*80)
    print("Testing Complete!")
    print("="*80)

Northwest Missouri State University - Catalog RAG Testing

🔧 Initializing RAG Tester...
✅ Connected to Pinecone index: salesassistai


OPTION 1: Single Question Test (Detailed)

🔍 QUESTION: What computer science programs are available?

📚 Searching for relevant information...
✅ Found 3 relevant documents

  📄 Source 1:
     Title: Program: Computer Science Comprehensive Major, 66-69 hours: B.S.-No Minor Requir...
     Type: program
     Relevance: 0.5858
     URL: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1374&returnto=177...

  📄 Source 2:
     Title: Program: Computer Science Comprehensive Major, 66-69 hours: B.S.-No Minor Requir...
     Type: program
     Relevance: 0.5847
     URL: https://catalog.nwmissouri.edu/preview_program.php?catoid=6&poid=1374...

  📄 Source 3:
     Title: Program: Computer Science Comprehensive Major, 66-69 hours: B.S.-No Minor Requir...
     Type: program
     Relevance: 0.5778
     URL: https://catalog.nwmissouri.edu/preview_program

KeyboardInterrupt: Interrupted by user

In [ ]:
Northwest A-Z HTML DataSets

In [ ]:
# ============================================================
# Northwest Missouri State University A-Z Directory Scraper
# Purpose: Scrape A-Z directory (depth 2 only) and store in Pinecone
# ============================================================

# Step 1: Install required packages
!pip install -q requests beautifulsoup4 openai pinecone-client lxml
!pip uninstall -y pinecone-client
!pip install -U pinecone

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs
import json
import time
import re
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from collections import deque
import hashlib

# ============================================================
# Enhanced A-Z Directory Scraper Class (Depth 2 Only)
# ============================================================
class AZDirectoryScraper:
    def __init__(self, base_url, max_depth=2):
        self.base_url = base_url
        self.max_depth = max_depth  # Fixed at 2
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        })
        self.visited_urls = set()
        self.all_data = []
        self.base_domain = urlparse(base_url).netloc

    def is_valid_internal_link(self, url):
        """Check if URL is a valid internal link"""
        try:
            parsed = urlparse(url)

            # Must be from same domain
            if parsed.netloc and parsed.netloc != self.base_domain:
                return False

            # Exclude file downloads
            excluded_extensions = ['.pdf', '.doc', '.docx', '.xls', '.xlsx',
                                  '.zip', '.jpg', '.png', '.gif', '.mp4']
            if any(url.lower().endswith(ext) for ext in excluded_extensions):
                return False

            # Exclude external links and non-content URLs
            excluded_patterns = ['mailto:', 'tel:', 'javascript:', '#',
                               'facebook.com', 'twitter.com', 'instagram.com',
                               'youtube.com', 'linkedin.com']
            if any(pattern in url.lower() for pattern in excluded_patterns):
                return False

            return True

        except Exception as e:
            print(f"Error validating URL {url}: {e}")
            return False

    def normalize_url(self, url):
        """Normalize URL to avoid duplicates"""
        parsed = urlparse(url)
        # Remove fragments
        normalized = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
        if parsed.query:
            normalized += f"?{parsed.query}"
        # Remove trailing slash for consistency
        if normalized.endswith('/') and normalized.count('/') > 3:
            normalized = normalized[:-1]
        return normalized

    def extract_links_from_page(self, url, soup):
        """Extract all valid internal links from a page"""
        links = set()

        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urljoin(url, href)
            normalized_url = self.normalize_url(full_url)

            if (self.is_valid_internal_link(normalized_url) and
                normalized_url not in self.visited_urls):
                links.add(normalized_url)

        return links

    def clean_text(self, text):
        """Clean and normalize text content"""
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        # Remove multiple newlines
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()

    def extract_page_content(self, url):
        """Extract structured content from a page"""
        try:
            response = self.session.get(url, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove unwanted elements
            for element in soup(['script', 'style', 'nav', 'header',
                               'footer', 'aside', 'iframe', 'noscript']):
                element.decompose()

            # Extract title
            title = soup.find('title')
            title_text = title.get_text(strip=True) if title else 'No Title'

            # Try to find main content area
            main_content = (
                soup.find('main') or
                soup.find('article') or
                soup.find('div', id='content') or
                soup.find('div', class_='content') or
                soup.find('div', id='main') or
                soup.body
            )

            if not main_content:
                return None

            # Extract text content
            text_content = main_content.get_text(separator='\n', strip=True)
            text_content = self.clean_text(text_content)

            # Extract all links for crawling
            links = self.extract_links_from_page(url, soup)

            # Extract headings for better structure
            headings = []
            for h in soup.find_all(['h1', 'h2', 'h3']):
                headings.append(h.get_text(strip=True))

            return {
                'url': url,
                'title': title_text,
                'content': text_content,
                'headings': headings,
                'content_length': len(text_content),
                'links': links
            }

        except Exception as e:
            print(f"❌ Error extracting content from {url}: {e}")
            return None

    def crawl_bfs_depth2(self, start_url):
        """Breadth-First Search crawling - ONLY DEPTH 0, 1, 2"""
        queue = deque([(start_url, 0)])  # (url, depth)
        self.visited_urls.add(self.normalize_url(start_url))

        print("🚀 Starting BFS crawl (Depth 0, 1, 2 only)...")
        print(f"📊 Max depth: {self.max_depth}")
        print(f"🔗 Starting URL: {start_url}\n")

        depth_stats = {0: 0, 1: 0, 2: 0}

        while queue:
            current_url, depth = queue.popleft()

            # CRITICAL: Stop if depth exceeds 2
            if depth > self.max_depth:
                continue

            print(f"{'  ' * depth}📄 [Depth {depth}] Crawling: {current_url[:100]}...")

            # Extract content from current page
            page_data = self.extract_page_content(current_url)

            if page_data and page_data['content_length'] > 100:
                # Store the page data
                self.all_data.append({
                    'id': len(self.all_data),
                    'url': page_data['url'],
                    'title': page_data['title'],
                    'content': page_data['content'],
                    'headings': page_data['headings'],
                    'content_length': page_data['content_length'],
                    'depth': depth,
                    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
                })

                depth_stats[depth] += 1

                print(f"{'  ' * depth}✅ Extracted {page_data['content_length']} chars")
                print(f"{'  ' * depth}🔗 Found {len(page_data['links'])} links")

                # IMPORTANT: Only add links to queue if current depth < 2
                # This means:
                # - Depth 0 (start page): Add links → they become depth 1
                # - Depth 1 pages: Add links → they become depth 2
                # - Depth 2 pages: DON'T add links (they would be depth 3)
                if depth < 2:
                    for link in page_data['links']:
                        normalized_link = self.normalize_url(link)
                        if normalized_link not in self.visited_urls:
                            self.visited_urls.add(normalized_link)
                            queue.append((link, depth + 1))
                            print(f"{'  ' * (depth+1)}➕ Queued for depth {depth + 1}: {link[:80]}")
                else:
                    print(f"{'  ' * depth}⏹️  Depth 2 reached - not following links")

            # Rate limiting
            time.sleep(0.5)

        print(f"\n{'='*60}")
        print(f"✅ Crawling completed!")
        print(f"📄 Total pages scraped: {len(self.all_data)}")
        print(f"📊 Pages by depth: Depth 0={depth_stats[0]}, Depth 1={depth_stats[1]}, Depth 2={depth_stats[2]}")
        print(f"{'='*60}\n")
        return self.all_data

    def save_to_jsonl(self, data, filename="az_directory_data.jsonl"):
        """Save scraped data to JSONL file"""
        with open(filename, 'w', encoding='utf-8') as f:
            for item in data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f"💾 Data saved to {filename}")

# ============================================================
# Pinecone Integration
# ============================================================
class PineconeRAG:
    def __init__(self, openai_api_key, pinecone_api_key, index_name="nw-az-directory"):
        self.client = OpenAI(api_key=openai_api_key)
        self.pc = Pinecone(api_key=pinecone_api_key)
        self.index_name = index_name
        self.index = None

    def create_or_connect_index(self, dimension=1536):
        """Create new index or connect to existing one"""
        try:
            # Try to connect to existing index
            self.index = self.pc.Index(self.index_name)
            print(f"✅ Connected to existing index: {self.index_name}")
        except:
            print(f"📝 Creating new index: {self.index_name}")
            self.pc.create_index(
                name=self.index_name,
                dimension=dimension,
                metric="cosine",
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1"
                )
            )
            time.sleep(10)  # Wait for index to initialize
            self.index = self.pc.Index(self.index_name)
            print(f"✅ Index created successfully")

    def chunk_text(self, text, chunk_size=1000, overlap=200):
        """Split text into overlapping chunks"""
        chunks = []
        start = 0

        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]

            # Try to break at sentence boundary
            if end < len(text):
                last_period = chunk.rfind('.')
                last_newline = chunk.rfind('\n')
                break_point = max(last_period, last_newline)

                if break_point > chunk_size * 0.5:  # At least 50% into chunk
                    chunk = chunk[:break_point + 1]
                    end = start + break_point + 1

            chunks.append(chunk.strip())
            start = end - overlap

        return chunks

    def generate_embedding(self, text):
        """Generate embedding using OpenAI"""
        try:
            response = self.client.embeddings.create(
                model="text-embedding-3-small",
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"❌ Error generating embedding: {e}")
            return None

    def insert_documents(self, documents, batch_size=100):
        """Insert documents into Pinecone with chunking"""
        print(f"\n🗄️ Inserting {len(documents)} documents into Pinecone...")

        successful_inserts = 0
        vectors_to_upsert = []

        for doc_idx, doc in enumerate(documents):
            try:
                # Chunk the content
                chunks = self.chunk_text(doc['content'])

                for chunk_idx, chunk in enumerate(chunks):
                    if len(chunk.strip()) < 50:  # Skip very small chunks
                        continue

                    # Prepare text for embedding
                    embedding_text = f"""
Title: {doc['title']}
Content: {chunk}
URL: {doc['url']}
                    """.strip()

                    # Generate embedding
                    embedding = self.generate_embedding(embedding_text)

                    if embedding:
                        # Create unique ID
                        unique_id = f"doc_{doc_idx}_chunk_{chunk_idx}_{hashlib.md5(chunk.encode()).hexdigest()[:8]}"

                        # Prepare metadata
                        metadata = {
                            'title': doc['title'][:500],  # Limit length
                            'url': doc['url'],
                            'content': chunk[:1000],  # Store chunk preview
                            'depth': doc['depth'],
                            'chunk_index': chunk_idx,
                            'total_chunks': len(chunks),
                            'timestamp': doc['timestamp']
                        }

                        # Add headings if available
                        if doc.get('headings'):
                            metadata['headings'] = ', '.join(doc['headings'][:5])[:500]

                        vectors_to_upsert.append({
                            'id': unique_id,
                            'values': embedding,
                            'metadata': metadata
                        })

                        successful_inserts += 1

                # Batch upsert
                if len(vectors_to_upsert) >= batch_size:
                    self.index.upsert(vectors=vectors_to_upsert)
                    print(f"  ✅ Inserted batch of {len(vectors_to_upsert)} vectors")
                    vectors_to_upsert = []

                if (doc_idx + 1) % 10 == 0:
                    print(f"  📊 Processed {doc_idx + 1}/{len(documents)} documents...")

            except Exception as e:
                print(f"  ❌ Error processing document {doc_idx}: {e}")
                continue

        # Insert remaining vectors
        if vectors_to_upsert:
            self.index.upsert(vectors=vectors_to_upsert)
            print(f"  ✅ Inserted final batch of {len(vectors_to_upsert)} vectors")

        print(f"\n✅ Total vectors inserted: {successful_inserts}")
        return successful_inserts

# ============================================================
# Main Execution
# ============================================================
if __name__ == "__main__":
    print("=" * 60)
    print("Northwest Missouri State University A-Z Directory Scraper")
    print("DEPTH LIMIT: 2 (Start page + linked pages + their links)")
    print("=" * 60)

    # Configuration
    BASE_URL = "https://www.nwmissouri.edu/az.htm"
    MAX_DEPTH = 2  # Fixed at 2

    # API Keys - REPLACE WITH YOUR ACTUAL KEYS
    OPENAI_API_KEY = ""
    PINECONE_API_KEY = ""
    INDEX_NAME = "salesassistai"

    # Validate API keys
    if "your-" in OPENAI_API_KEY or "your-" in PINECONE_API_KEY:
        print("⚠️  WARNING: Please replace the API keys with your actual keys!")
        print("Update OPENAI_API_KEY and PINECONE_API_KEY variables.")
    else:
        # Step 1: Scrape the A-Z directory (depth 2 only)
        scraper = AZDirectoryScraper(BASE_URL, max_depth=MAX_DEPTH)
        scraped_data = scraper.crawl_bfs_depth2(BASE_URL)

        # Save raw data
        scraper.save_to_jsonl(scraped_data, "az_directory_raw_data.jsonl")

        # Step 2: Insert into Pinecone
        rag = PineconeRAG(OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME)
        rag.create_or_connect_index()

        successful_inserts = rag.insert_documents(scraped_data)

        # Final Summary
        print("\n" + "=" * 60)
        print("📊 FINAL SUMMARY")
        print("=" * 60)
        print(f"✅ Total pages scraped: {len(scraped_data)}")
        print(f"✅ Total vectors inserted: {successful_inserts}")
        print(f"✅ Index name: {INDEX_NAME}")
        print(f"✅ Raw data saved: az_directory_raw_data.jsonl")
        print(f"✅ Crawl depth: 0, 1, 2 only")
        print("=" * 60)

Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


Northwest Missouri State University A-Z Directory Scraper
DEPTH LIMIT: 2 (Start page + linked pages + their links)
🚀 Starting BFS crawl (Depth 0, 1, 2 only)...
📊 Max depth: 2
🔗 Starting URL: https://www.nwmissouri.edu/az.htm

📄 [Depth 0] Crawling: https://www.nwmissouri.edu/az.htm...
✅ Extracted 10998 chars
🔗 Found 234 links
  ➕ Queued for depth 1: https://www.nwmissouri.edu/giving/foundation/board/index.htm
  ➕ Queued for depth 1: https://www.nwmissouri.edu/academics/undergraduate/majors/pre-osteopathy.htm
  ➕ Queued for depth 1: https://www.nwmissouri.edu/academics/undergraduate/majors/political-science.htm
  ➕ Queued for depth 1: https://www.nwmissouri.edu/studentaccounts/tuitionandfees.htm
  ➕ Queued for depth 1: https://www.nwmissouri.edu/alumni/missing/index.htm
  ➕ Queued for depth 1: https://www.nwmissouri.edu/academics/undergraduate/majors/pre-respiratory-therap
  ➕ Queued for depth 1: https://www.nwmissouri.edu/wellness/Hope4All/index.htm
  ➕ Queued for depth 1: https://www.n

/tmp/ipython-input-2065820169.py:105: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.content, 'html.parser')


  📄 [Depth 1] Crawling: https://www.nwmissouri.edu/academics/undergraduate/majors/writing.htm...
  ✅ Extracted 4189 chars
  🔗 Found 1 links
    ➕ Queued for depth 2: https://www.nwmissouri.edu/academics/undergraduate/certificates/professional-wri
  📄 [Depth 1] Crawling: https://www.nwmissouri.edu/academics/honors/index.htm...
  ✅ Extracted 2386 chars
  🔗 Found 4 links
    ➕ Queued for depth 2: https://www.nwmissouri.edu/academics/honors/events.htm
    ➕ Queued for depth 2: https://www.nwmissouri.edu/academics/honors/about.htm
    ➕ Queued for depth 2: https://www.nwmissouri.edu/academics/honors/benefits.htm
    ➕ Queued for depth 2: https://www.nwmissouri.edu/academics/honors/eligibility.htm
  📄 [Depth 1] Crawling: https://www.nwmissouri.edu/maps/index.htm...
  ✅ Extracted 670 chars
  🔗 Found 5 links
    ➕ Queued for depth 2: https://www.nwmissouri.edu/map/tours/dining.htm
    ➕ Queued for depth 2: https://www.nwmissouri.edu/map/tours/recreation.htm
    ➕ Queued for depth 2: https://ww

    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/giving/opportunities/projects/martindale/index.htm...
    ✅ Extracted 2627 chars
    🔗 Found 7 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/health/about.htm...
    ✅ Extracted 2048 chars
    🔗 Found 7 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/health/EIM-OC.htm...
    ✅ Extracted 3387 chars
    🔗 Found 0 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/health/directory/index.htm...
    ✅ Extracted 3930 chars
    🔗 Found 36 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/studentaffairs/ssenate/forms/index.htm...
    ✅ Extracted 1046 chars
    🔗 Found 0 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/housing/current/handbook/index.htm...
    ✅ Extra

    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/finearts/music/media/towerchoir/02_Track_2.mp3...


    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/finearts/music/media/towerchoir/05_Track_5.mp3...


    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/finearts/music/media/towerchoir/04_Track_4.mp3...


    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/finearts/music/media/towerchoir/01_Track_1.mp3...


    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/academics/undergraduate/minors/agricultural-communications.htm...
    ✅ Extracted 1613 chars
    🔗 Found 0 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/academics/undergraduate/majors/radiological-science.htm...
    ✅ Extracted 2396 chars
    🔗 Found 0 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/services/purchasing/bidlisting.htm...
    ✅ Extracted 536 chars
    🔗 Found 0 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/services/purchasing/mailcopy/index.htm...
    ✅ Extracted 5457 chars
    🔗 Found 0 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] Crawling: https://www.nwmissouri.edu/housing/current/services/student-packages.htm...
    ✅ Extracted 3595 chars
    🔗 Found 1 links
    ⏹️  Depth 2 reached - not following links
    📄 [Depth 2] C

In [ ]:
Faculty and Staff information

In [ ]:
# ============================================================
# Northwest Missouri State University Multi-Directory Scraper
# Purpose: Scrape multiple directory pages (depth 3) and store in Pinecone
# ============================================================

# Step 1: Install required packages
!pip install -q requests beautifulsoup4 openai pinecone-client lxml
!pip uninstall -y pinecone-client
!pip install -U pinecone

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json
import time
import re
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from collections import deque
import hashlib

# ============================================================
# Enhanced Multi-URL Directory Scraper (Depth 3)
# ============================================================
class MultiDirectoryScraper:
    def __init__(self, start_urls, max_depth=1):
        self.start_urls = start_urls
        self.max_depth = max_depth
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        self.visited_urls = set()
        self.all_data = []
        self.base_domain = "nwmissouri.edu"

    def is_valid_internal_link(self, url):
        """Check if URL is valid internal link"""
        try:
            parsed = urlparse(url)

            # Must be from nwmissouri.edu domain
            if parsed.netloc and self.base_domain not in parsed.netloc:
                return False

            # Exclude file downloads
            excluded_extensions = ['.pdf', '.doc', '.docx', '.xls', '.xlsx',
                                  '.zip', '.jpg', '.png', '.gif', '.mp4', '.ppt']
            if any(url.lower().endswith(ext) for ext in excluded_extensions):
                return False

            # Exclude external links
            excluded_patterns = ['mailto:', 'tel:', 'javascript:', '#',
                               'facebook.com', 'twitter.com', 'instagram.com',
                               'youtube.com', 'linkedin.com']
            if any(pattern in url.lower() for pattern in excluded_patterns):
                return False

            return True

        except Exception as e:
            print(f"Error validating URL {url}: {e}")
            return False

    def normalize_url(self, url):
        """Normalize URL to avoid duplicates"""
        parsed = urlparse(url)
        normalized = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
        if parsed.query:
            normalized += f"?{parsed.query}"
        if normalized.endswith('/') and normalized.count('/') > 3:
            normalized = normalized[:-1]
        return normalized

    def extract_links_from_page(self, url, soup):
        """Extract all valid internal links from a page"""
        links = set()

        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urljoin(url, href)
            normalized_url = self.normalize_url(full_url)

            if (self.is_valid_internal_link(normalized_url) and
                normalized_url not in self.visited_urls):
                links.add(normalized_url)

        return links

    def clean_text(self, text):
        """Clean and normalize text content"""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()

    def extract_contact_info(self, soup):
        """Extract contact information (email, phone) from page"""
        contacts = {
            'emails': [],
            'phones': []
        }

        # Find emails
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        page_text = soup.get_text()
        emails = re.findall(email_pattern, page_text)
        contacts['emails'] = list(set(emails))[:5]  # Limit to 5 unique emails

        # Find phone numbers
        phone_pattern = r'\b(?:\+?1[-.]?)?\(?([0-9]{3})\)?[-.]?([0-9]{3})[-.]?([0-9]{4})\b'
        phones = re.findall(phone_pattern, page_text)
        contacts['phones'] = [f"({p[0]}) {p[1]}-{p[2]}" for p in phones[:5]]

        return contacts

    def extract_page_content(self, url):
        """Extract structured content from a page"""
        try:
            response = self.session.get(url, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove unwanted elements
            for element in soup(['script', 'style', 'nav', 'header',
                               'footer', 'aside', 'iframe', 'noscript']):
                element.decompose()

            # Extract title
            title = soup.find('title')
            title_text = title.get_text(strip=True) if title else 'No Title'

            # Try to find main content area
            main_content = (
                soup.find('main') or
                soup.find('article') or
                soup.find('div', id='content') or
                soup.find('div', class_='content') or
                soup.find('div', id='main') or
                soup.body
            )

            if not main_content:
                return None

            # Extract text content
            text_content = main_content.get_text(separator='\n', strip=True)
            text_content = self.clean_text(text_content)

            # Extract links
            links = self.extract_links_from_page(url, soup)

            # Extract headings
            headings = []
            for h in soup.find_all(['h1', 'h2', 'h3']):
                headings.append(h.get_text(strip=True))

            # Extract contact info
            contacts = self.extract_contact_info(soup)

            return {
                'url': url,
                'title': title_text,
                'content': text_content,
                'headings': headings,
                'emails': contacts['emails'],
                'phones': contacts['phones'],
                'content_length': len(text_content),
                'links': links
            }

        except Exception as e:
            print(f"❌ Error extracting content from {url}: {e}")
            return None

    def crawl_bfs_depth3(self):
        """Breadth-First Search crawling - Depth 0, 1, 2, 3"""
        # Initialize queue with all start URLs at depth 0
        queue = deque()
        for url in self.start_urls:
            normalized = self.normalize_url(url)
            if normalized not in self.visited_urls:
                queue.append((url, 0))
                self.visited_urls.add(normalized)

        print("🚀 Starting Multi-URL BFS crawl (Depth 0-3)...")
        print(f"📊 Max depth: {self.max_depth}")
        print(f"🔗 Starting with {len(self.start_urls)} URLs\n")

        depth_stats = {0: 0, 1: 0, 2: 0, 3: 0}
        start_time = time.time()

        while queue:
            current_url, depth = queue.popleft()

            # Stop if depth exceeds 3
            if depth > self.max_depth:
                continue

            print(f"{'  ' * depth}📄 [Depth {depth}] Crawling: {current_url[:80]}...")

            # Extract content from current page
            page_data = self.extract_page_content(current_url)

            if page_data and page_data['content_length'] > 100:
                # Store the page data
                self.all_data.append({
                    'id': len(self.all_data),
                    'url': page_data['url'],
                    'title': page_data['title'],
                    'content': page_data['content'],
                    'headings': page_data['headings'],
                    'emails': page_data['emails'],
                    'phones': page_data['phones'],
                    'content_length': page_data['content_length'],
                    'depth': depth,
                    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
                })

                depth_stats[depth] += 1

                print(f"{'  ' * depth}✅ Extracted {page_data['content_length']} chars")
                print(f"{'  ' * depth}📧 Emails: {len(page_data['emails'])}, 📞 Phones: {len(page_data['phones'])}")
                print(f"{'  ' * depth}🔗 Found {len(page_data['links'])} links")

                # Add links to queue if not at max depth
                if depth < self.max_depth:
                    for link in page_data['links']:
                        normalized_link = self.normalize_url(link)
                        if normalized_link not in self.visited_urls:
                            self.visited_urls.add(normalized_link)
                            queue.append((link, depth + 1))
                else:
                    print(f"{'  ' * depth}⏹️  Max depth reached - not following links")

            # Rate limiting
            time.sleep(0.5)

            # Progress update every 50 pages
            if len(self.all_data) % 50 == 0 and len(self.all_data) > 0:
                elapsed = time.time() - start_time
                print(f"\n⏱️  Progress: {len(self.all_data)} pages in {elapsed:.1f}s\n")

        print(f"\n{'='*70}")
        print(f"✅ Crawling completed!")
        print(f"📄 Total pages scraped: {len(self.all_data)}")
        print(f"📊 Pages by depth: D0={depth_stats[0]}, D1={depth_stats[1]}, D2={depth_stats[2]}, D3={depth_stats[3]}")
        print(f"⏱️  Total time: {time.time() - start_time:.1f}s")
        print(f"{'='*70}\n")
        return self.all_data

    def save_to_jsonl(self, data, filename="nw_directory_depth3.jsonl"):
        """Save scraped data to JSONL file"""
        with open(filename, 'w', encoding='utf-8') as f:
            for item in data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f"💾 Data saved to {filename}")

# ============================================================
# Pinecone Integration
# ============================================================
class PineconeRAG:
    def __init__(self, openai_api_key, pinecone_api_key, index_name="nw-directory"):
        self.client = OpenAI(api_key=openai_api_key)
        self.pc = Pinecone(api_key=pinecone_api_key)
        self.index_name = index_name
        self.index = None

    def create_or_connect_index(self, dimension=1536):
        """Create new index or connect to existing one"""
        try:
            self.index = self.pc.Index(self.index_name)
            print(f"✅ Connected to existing index: {self.index_name}")
        except:
            print(f"📝 Creating new index: {self.index_name}")
            self.pc.create_index(
                name=self.index_name,
                dimension=dimension,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1")
            )
            time.sleep(10)
            self.index = self.pc.Index(self.index_name)
            print(f"✅ Index created successfully")

    def chunk_text(self, text, chunk_size=1000, overlap=200):
        """Split text into overlapping chunks"""
        chunks = []
        start = 0

        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]

            if end < len(text):
                last_period = chunk.rfind('.')
                last_newline = chunk.rfind('\n')
                break_point = max(last_period, last_newline)

                if break_point > chunk_size * 0.5:
                    chunk = chunk[:break_point + 1]
                    end = start + break_point + 1

            chunks.append(chunk.strip())
            start = end - overlap

        return chunks

    def generate_embedding(self, text):
        """Generate embedding using OpenAI"""
        try:
            response = self.client.embeddings.create(
                model="text-embedding-3-small",
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"❌ Error generating embedding: {e}")
            return None

    def insert_documents(self, documents, batch_size=100):
        """Insert documents into Pinecone with chunking"""
        print(f"\n🗄️ Inserting {len(documents)} documents into Pinecone...")

        successful_inserts = 0
        vectors_to_upsert = []

        for doc_idx, doc in enumerate(documents):
            try:
                chunks = self.chunk_text(doc['content'])

                for chunk_idx, chunk in enumerate(chunks):
                    if len(chunk.strip()) < 50:
                        continue

                    # Prepare text for embedding
                    embedding_text = f"""
Title: {doc['title']}
Content: {chunk}
URL: {doc['url']}
Emails: {', '.join(doc.get('emails', []))}
Phones: {', '.join(doc.get('phones', []))}
                    """.strip()

                    embedding = self.generate_embedding(embedding_text)

                    if embedding:
                        unique_id = f"doc_{doc_idx}_chunk_{chunk_idx}_{hashlib.md5(chunk.encode()).hexdigest()[:8]}"

                        metadata = {
                            'title': doc['title'][:500],
                            'url': doc['url'],
                            'content': chunk[:1000],
                            'depth': doc['depth'],
                            'chunk_index': chunk_idx,
                            'total_chunks': len(chunks),
                            'timestamp': doc['timestamp']
                        }

                        if doc.get('headings'):
                            metadata['headings'] = ', '.join(doc['headings'][:5])[:500]

                        if doc.get('emails'):
                            metadata['emails'] = ', '.join(doc['emails'][:5])[:500]

                        if doc.get('phones'):
                            metadata['phones'] = ', '.join(doc['phones'][:5])[:500]

                        vectors_to_upsert.append({
                            'id': unique_id,
                            'values': embedding,
                            'metadata': metadata
                        })

                        successful_inserts += 1

                if len(vectors_to_upsert) >= batch_size:
                    self.index.upsert(vectors=vectors_to_upsert)
                    print(f"  ✅ Inserted batch of {len(vectors_to_upsert)} vectors")
                    vectors_to_upsert = []

                if (doc_idx + 1) % 10 == 0:
                    print(f"  📊 Processed {doc_idx + 1}/{len(documents)} documents...")

            except Exception as e:
                print(f"  ❌ Error processing document {doc_idx}: {e}")
                continue

        if vectors_to_upsert:
            self.index.upsert(vectors=vectors_to_upsert)
            print(f"  ✅ Inserted final batch of {len(vectors_to_upsert)} vectors")

        print(f"\n✅ Total vectors inserted: {successful_inserts}")
        return successful_inserts

# ============================================================
# Main Execution
# ============================================================
if __name__ == "__main__":
    print("=" * 70)
    print("Northwest Missouri State University Directory Scraper")
    print("DEPTH LIMIT: 3 | Multiple Starting URLs")
    print("=" * 70)

    # All directory URLs to scrape
    START_URLS = [
        "https://www.nwmissouri.edu/socialsciences/directory/index.htm",
        "https://www.nwmissouri.edu/csis/directory/index.htm",
        "https://www.nwmissouri.edu/education/directory/index.htm",
        "https://www.nwmissouri.edu/naturalsciences/directory/index.htm",
        "https://www.nwmissouri.edu/business/directory/index.htm",
        "https://www.nwmissouri.edu/health/directory/index.htm",
        "https://www.nwmissouri.edu/ag/directory/index.htm",
        "https://www.nwmissouri.edu/llw/directory/index.htm"
    ]

    MAX_DEPTH = 3

    # API Keys - REPLACE WITH YOUR ACTUAL KEYS
    OPENAI_API_KEY = ""
    PINECONE_API_KEY = ""
    INDEX_NAME = "salesassistai"

    # Step 1: Scrape all directories (depth 3)
    scraper = MultiDirectoryScraper(START_URLS, max_depth=MAX_DEPTH)
    scraped_data = scraper.crawl_bfs_depth3()

    # Save raw data
    scraper.save_to_jsonl(scraped_data, "nw_directory_depth3_raw.jsonl")

    # Step 2: Insert into Pinecone
    rag = PineconeRAG(OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME)
    rag.create_or_connect_index()

    successful_inserts = rag.insert_documents(scraped_data)

    # Final Summary
    print("\n" + "=" * 70)
    print("📊 FINAL SUMMARY")
    print("=" * 70)
    print(f"✅ Total pages scraped: {len(scraped_data)}")
    print(f"✅ Total vectors inserted: {successful_inserts}")
    print(f"✅ Starting URLs: {len(START_URLS)}")
    print(f"✅ Index name: {INDEX_NAME}")
    print(f"✅ Raw data saved: nw_directory_depth3_raw.jsonl")
    print(f"✅ Crawl depth: 0-3")
    print("=" * 70)

Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
Northwest Missouri State University Directory Scraper
DEPTH LIMIT: 3 | Multiple Starting URLs
🚀 Starting Multi-URL BFS crawl (Depth 0-3)...
📊 Max depth: 3
🔗 Starting with 8 URLs

📄 [Depth 0] Crawling: https://www.nwmissouri.edu/socialsciences/directory/index.htm...
✅ Extracted 2328 chars
📧 Emails: 5, 📞 Phones: 0
🔗 Found 31 links
📄 [Depth 0] Crawling: https://www.nwmissouri.edu/csis/directory/index.htm...
✅ Extracted 1649 chars
📧 Emails: 5, 📞 Phones: 0
🔗 Found 18 links
📄 [Depth 0] Crawling: https://www.nwmissouri.edu/education/directory/index.htm...
✅ Extracted 2275 chars
📧 Emails: 5, 📞 Phones: 0
🔗 Found 24 links
📄 [Depth 0] Crawling: https://www.nwmissouri.edu/naturalsciences/directory/index.htm...
✅ Extracted 2982 chars
📧 Emails: 5, 📞 Phones: 0
🔗 Found 32 links
📄 [Depth 0] Crawling: https://www.nwmissouri.edu/business/directory/index.htm...
✅ Extracte